<a href="https://colab.research.google.com/github/egotist0/EvaDB_Proj/blob/master/News_Recommendation_Platform_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet "evadb[vision,document,notebook]"
%pip install --quiet "pinecone_client"
%pip install --quiet "kornia"
%pip install --quiet "swifter"
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.1/530.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 69.2 MB/s et

In [2]:
import kornia
import evadb
cursor = evadb.connect().cursor()
import warnings
warnings.filterwarnings("ignore")
import os
import json
import pandas as pd
import pinecone
import re
import requests
from sentence_transformers import SentenceTransformer
from statistics import mean
import swifter
import openai

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 738kB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 156MB/s]


In [3]:
# Set api key
api_key = ""
os.environ["PINECONE_API_KEY"] = api_key

openai.api_key = ""

# Set environment
environment = ""
os.environ["PINECONE_ENV"] = environment

PINECONE_INDEX_NAME = "article-recommendation-service"
DATA_FILE = "articles.csv"

In [4]:
pinecone.init(api_key=api_key, environment=environment)
if PINECONE_INDEX_NAME in pinecone.list_indexes():
        pinecone.delete_index(PINECONE_INDEX_NAME)
pinecone.create_index(dimension=300, name=PINECONE_INDEX_NAME, metric="cosine", shards=1)
pinecone_index = pinecone.Index(index_name=PINECONE_INDEX_NAME)

model = SentenceTransformer('average_word_embeddings_komninos')

In [5]:
data = pd.read_csv("articles.csv")

# prepare the data
# rename id column and remove unnecessary columns
data.rename(columns={"Unnamed: 0": "article_id"}, inplace = True)
data.drop(columns=['date'], inplace = True)

# extract only first few sentences of each article for quicker vector calculations
data['content'] = data['content'].fillna('')
data['content'] = data.content.swifter.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:4]))
data['title_and_content'] = data['title'] + ' ' + data['content']

# create a vector embedding based on title and article columns
encoded_articles = model.encode(data['title_and_content'], show_progress_bar=True)
data['article_vector'] = pd.Series(encoded_articles.tolist())


# upload the data into pinecone
upsert_batch = []
for i, row in data.iterrows():
    upsert_batch.append((str(row.id), row.article_vector))

    if len(upsert_batch) > 500:
        pinecone_index.upsert(upsert_batch)
        upsert_batch = []

# Process any remaining data in upsert_batch
if upsert_batch:
    pinecone_index.upsert(upsert_batch)


Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [6]:
titles_mapped=dict(zip(data.id, data.title))
publications_mapped=dict(zip(data.id, data.publication))
content_mapped=dict(zip(data.id, data.content))

In [7]:
def article_recomendation(artile_list):
    meta_list = data.loc[data['id'].isin(artile_list)]

    article_vectors = meta_list['article_vector']
    article_vector = [*map(mean, zip(*article_vectors))]

    query_results = pinecone_index.query(vector=article_vector, top_k=10)
    res = query_results['matches']

    results_list = []

    for idx, item in enumerate(res):
        results_list.append({
            "title": titles_mapped[int(item.id)],
            "publication": publications_mapped[int(item.id)],
            "content": content_mapped[int(item.id)],
        })

    return results_list

In [18]:
# Person whose interest lies in computer science
health_title_df = data[data['title'].str.contains('Software Engineer')].head(5)
# get id list
id_list = health_title_df['id'].tolist()

# get title list
original_articles = []
for idx, item in enumerate(id_list):
  original_articles.append({
      "title": titles_mapped[int(item)],
      "publication": publications_mapped[int(item)],
      "content": content_mapped[int(item)],
      })

recommendation_articles = article_recomendation(id_list)

# Evaluation
content = "I will provide the titles and contents of five articles I have read before, as well as 10 articles recommended to me. Please evaluate the similarity in themes and relevance of these 10 articles to my previously read articles.\n"
original = "history articles: "+str(original_articles) + "\n"
recommendation = "recommendation articles:"+str(recommendation_articles)

completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", \
                                       messages=[{"role": "user", "content": content + original + recommendation}])

# print the completion
print(completion.choices[0].message.content)


The previously read article is about a report stating that only 36 percent of Indian software engineers can write useful code. The article discusses the findings of a study conducted by Aspiring Minds, an Indian skills assessment company, which assessed the programming skills of Indian software engineers using an automated tool called Automata. The study found that only a low percentage of engineers were able to write working, compilable code.

The recommended articles cover a range of topics, but there are several that are related to technology, programming, and education. One article discusses Apple's free coding education app called Swift Playgrounds, which aims to teach children coding using the Swift programming language. Another article explores the topic of online dating and how it can lead to the lowering of standards. There are also articles about Apple's investments in artificial intelligence and a study that claims children are more proficient in operating smartphones than c

In [11]:
# Person whose interest lies in cancer
health_title_df = data[data['title'].str.contains('cancer')].head(5)
# get id list
id_list = health_title_df['id'].tolist()

# get title list
original_articles = []
for idx, item in enumerate(id_list):
  original_articles.append({
      "title": titles_mapped[int(item)],
      "publication": publications_mapped[int(item)],
      "content": content_mapped[int(item)],
      })

recommendation_articles = article_recomendation(id_list)

# Evaluation
content = "I will provide the titles and contents of five articles I have read before, as well as 10 articles recommended to me. Please evaluate the similarity in themes and relevance of these 10 articles to my previously read articles.\n"
original = "history articles: "+str(original_articles) + "\n"
recommendation = "recommendation articles:"+str(recommendation_articles)

completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", \
                                       messages=[{"role": "user", "content": content + original + recommendation}])

# print the completion
print(completion.choices[0].message.content)


The themes of the previously read articles are focused on various types of cancer, including breast cancer, cervical cancer, and skin cancer. They also highlight issues such as cancer mortality rates, racial disparities in cancer deaths, and the use of AI technology in cancer diagnosis.

The recommended articles cover similar themes, with a focus on cancer research and prevention. They discuss topics such as the impact of a high-fat diet on prostate cancer, the effectiveness of ovarian cancer prevention surgery, the racial gap in cervical cancer deaths, the use of AI in detecting skin cancer, and the ability to identify breast cancer patients who can skip chemotherapy through gene testing.

Overall, the recommended articles are highly relevant to the previously read articles, as they all revolve around cancer research, prevention, and treatment. The themes explored in the recommended articles align closely with the themes of the previously read articles, indicating a similarity of inte